# Neural Networks: There and Back Again...and Again...and Again

One of the most difficult tasks in machine learning is feature engineering. Feature engineering is where you create new features from the ones you have through mathematical operations such as multiplication. For example, making $x^2$ from $x$. This is done because many features end up being correlated and have nonlinear relationships with the output. Neural networks are useful because they automate feature engineering in a simple and straightforward way.

<img src="http://cs231n.github.io/assets/nn1/neural_net2.jpeg">
Taken from the MIT CS231n <a href="http://cs231n.github.io/neural-networks-1/">course page</a>. <a href="https://github.com/cs231n/cs231n.github.io/blob/master/LICENSE">LICENSE</a>.

Each circle in the diagram is called a neuron or node. Each feature in your input data gets a node. Each class in your output gets a node. In the above picture, there are two outputs, so it's a binary classification problem. To get from the input to the output you go through any number of hidden layers with any number of nodes; the numbers are hyperparameters. You can see that all the input nodes contribute to each hidden layer node somehow. The weights for how much each input contributes are the parameters to be optimized. It's called a hidden layer because the user doesn't get to see the parameters without purposefully looking for them. Thus, feature engineering is automated.

Neural nets are mainly used for classification, so that's the problem I'm going to focus on. I'll explain how it can be modified later.

## Matrix Multiplication: A Review

When trying to figure out how to code an algorithm, it is helpful to keep track of all the dimensions of the matrices and vectors you're working with. Remember what happens with the dimensions:
$$ [n \times m][m \times p] = [n \times p] $$
The inner two dimensions have to be the same; the operation is not defined otherwise. The product matrix then takes the outer dimensions. For example, take these two vectors (1D matrices):
$$ \mathbf{A} =  \begin{bmatrix}
                    a_1 \newline a_2 \newline a_3
                 \end{bmatrix},
\mathbf{B} = \begin{bmatrix}
                b_1 \newline b_2 \newline b_3
             \end{bmatrix} $$
They are both $[3 \times 1]$ vectors. They can't be multiplied together as is, but if you take the transpose of one, they can be. This gives us two possibilities:
$$ \mathbf{A^TB} = a_1b_1 + a_2b_2 + a_3b_3 \\
   \mathbf{AB^T} = \begin{bmatrix}
                        a_1b_1 & a_1b_2 & a_1b_3 \\
                        a_2b_1 & a_2b_2 & a_2b_3 \\
                        a_3b_1 & a_3b_2 & a_3b_3
                   \end{bmatrix} \\ $$
Notice that the answers are quite different.

This is what the above looks like as a summation:
$$ (\mathbf{AB})_{ij} = \sum_{k=1}^m A_{ik}B_{kj} $$
where $i$ goes from 1 to $n$ and $j$ goes from 1 to $p$.

## Binary Logistic Regression: A Review

First, let's review the math behind binary logistic regression. I'll be using the notation from <a href="https://www.coursera.org/learn/machine-learning/home/welcome">Machine Learning by Andrew Ng</a> on Coursera, except for two changes. One, I'm going to treat the parameter and feature vectors as row vectors, while Andrew Ng treats them as column vectors. Two, I'm going to keep the bias terms as a separate vector.

So, we have a binary classification problem with classes 0 and 1. You have a hypothesis about your data, $ h_{\theta}(x) $, which is the probability that the item with feature $x$ is in class 1. The probability of class 0 is $1 - h_{\theta}(x) $. What does this hypothesis look like? We need a function that can take any real number and squish it into the interval [0,1]. This is called the activation function. One such function is the sigmoid, or logistic, function:
$$ g(z) = \frac {1}{1+e^{-z}} $$
<img src="http://cs231n.github.io/assets/nn1/sigmoid.jpeg">
Taken from the MIT CS231n <a href="http://cs231n.github.io/neural-networks-1/">course page</a>. <a href="https://github.com/cs231n/cs231n.github.io/blob/master/LICENSE">LICENSE</a>.

When $ h_{\theta}(x) = g(z) \geq 0.5$, we predict class 1. When $ h_{\theta}(x) = g(z) < 0.5$, we predict class 0. Well, what is $z$? If we have $n$ features:
$$ z = b + \theta_1x_1 + ... + \theta_nx_n = b + \begin{bmatrix}x_1 &  x_2 &  ...  \hspace{2em}  x_n\end{bmatrix}\begin{bmatrix} \theta_1 \newline \theta_2 \newline \vdots \newline \theta_n\end{bmatrix}= b + \boldsymbol{x \theta}^T $$
where $\theta$ and $x$ are column vectors. $\theta$ are the parameters to optimize. $b$ is called the bias term and it acts much like an intercept term in linear regression. It allows you to shift the activation function to get a better fit.

The cost function for this problem is:
$$ J(\boldsymbol{\theta}) = - \frac{1}{m} \displaystyle \sum_{i=1}^m [y^{(i)}\log (h_\theta (\mathbf{x}^{(i)})) + (1 - y^{(i)})\log (1 - h_\theta(\mathbf{x}^{(i)}))] $$
where $m$ is the number of observations. Basically, if you think about a DataFrame, $i$ is the row and $m$ is the total number of rows. Then $\mathbf{x}$ is $[m \times n]$ and $\mathbf{y}$ is $[m \times 1]$. In vectorized notation, this is:
$$ J(\boldsymbol{\theta}) = - \frac{1}{m} [\mathbf{y}^T \log (g(b+\boldsymbol{x \theta}^T)) + (\mathbf{1 - y}^T) \log (g(1 - b+\boldsymbol{x \theta}^T))] $$

You can also add a regularization term:
$$ J(\boldsymbol{\theta}) = - \frac{1}{m} \displaystyle \sum_{i=1}^m [y^{(i)}\log (h_\theta (\mathbf{x}^{(i)})) + (1 - y^{(i)})\log (1 - h_\theta(\mathbf{x}^{(i)}))] + \frac{\lambda}{2m} \sum^{n}_{j=1} \theta_j$$  

And in vector notation:
$$ J(\boldsymbol{\theta}) = - \frac{1}{m} [\mathbf{y}^T \log (g(b+\boldsymbol{x \theta}^T)) + (\mathbf{1 - y}^T) \log (g(1 - b+\boldsymbol{x \theta}^T))] + \frac{\lambda}{2m}\boldsymbol\theta\boldsymbol\theta^T$$

Then you can minimize the cost function with the solver of your choice. Often times, the solver asks for a cost function and a derivative, so here's the derivative for all $\boldsymbol{\theta}$'s:
$$ \frac{\partial}{\partial\boldsymbol{\theta}} J(\boldsymbol{\theta}) = \frac{1}{m} \sum_{i=1}^m  (h_\theta(\mathbf{x}^{(i)}) - y^{(i)}) \mathbf{x}_j^{(i)} \\
= \frac{1}{m} \mathbf{x}^T(g(b+\boldsymbol{x \theta}^T) - \mathbf{y})
$$
And here's the derivative for the bias term: 
$$ \frac{\partial}{\partial b} J(\boldsymbol{\theta}) = \frac{1}{m} \sum_{i=1}^m  (h_\theta(\mathbf{x}^{(i)}) - y^{(i)}) \mathbf{x}_j^{(i)} \\
= \frac{1}{m} (g(b+\boldsymbol{x \theta}^T) - \mathbf{y})
$$
To add regularization to this, add $ \frac {\lambda}{m}\theta_j $ to the jth $\boldsymbol{\theta}$ derivative. Note that the bias term doesn't have regularization, so its derivative doesn't add regularization either.

The easiest solver to use is gradient descent and it's exactly what it sounds like. If you imagine a graph of the cost function vs the parameters, the minimum is the bottom of the valley in the graph. You have to descend the slope, or gradient, to get there.

<img src="https://upload.wikimedia.org/wikipedia/commons/5/5b/Gradient_descent_method.png">
By Роман Сузи (Сгенерирован программой, повернут вручную) [GFDL (http://www.gnu.org/copyleft/fdl.html) or CC BY-SA 3.0 (http://creativecommons.org/licenses/by-sa/3.0)], via Wikimedia Commons

To implement the algorithm, the estimates for $\boldsymbol\theta$ are updated according to their gradients, the cost function is recalculated, and this is repeated until convergence. Here are the equations for updating $\boldsymbol\theta$:
$$ \theta_j := \theta_j - \alpha \frac{\partial}{\partial \theta_j} J(\boldsymbol\theta) $$
$\alpha$ is called the learning rate, and it's a hyperparameter that determines how fast the algorithm converges. If it is too large, the algorithm could keep missing the minimum and never converge. If it's too small, the algorithm could take a long time to converge.

## Breakfast: Neural Nets from scratch
Neural nets just apply logistic regression at every neuron past the input layer. To keep things simple, I'm going to stick with a binary classification problem modified from <a href="http://www.wildml.com/2015/09/implementing-a-neural-network-from-scratch/">this</a> blog post. So, let's use scikit learn's <a href="http://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_moons.html">```make_moons()```</a> function to get some binary data.

In [ ]:
%matplotlib inline

In [ ]:
import sklearn.datasets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.rcParams['figure.figsize'] = [9,6]

In [ ]:
# Generate a dataset
np.random.seed(0)
X, y = sklearn.datasets.make_moons(200, noise=0.20)

In [ ]:
m, n = X.shape
X.shape

So we have 200 observations and 2 features.

In [ ]:
y.shape

Note that numpy doesn't distinguish between row and column vectors by default. Everything is just a 1D array. This could get confusing when delving into the math, so let's turn it into a column vector.

In [ ]:
y = np.reshape(y, (200,1))
y.shape

In [ ]:
# here's what it might look like in a DataFrame
data = np.column_stack((X, y))
data = pd.DataFrame(data, columns=["Height", "Daily Caloric Intake", "Hobbit?"])
data["Hobbit?"] = data["Hobbit?"].astype(int)
data.head()

In [ ]:
#plot it
plt.scatter(X[:,0], X[:,1], s=40, c=y, cmap=plt.cm.Spectral)

From the graph, it's easy to see that the division between hobbits and not hobbits is not linear, so logistic regression by itself won't work without some feature engineering. That's why we throw it into a neural net and let it do the engineering for us.

### Forward propagation
First, we start with our inputs and calculate forward to the output neurons using initial guesses for all $\boldsymbol\theta$'s. In our example, we'll have one hidden layer with 5 neurons. 
<img src="nn-from-scratch-3-layer-network.png">

For clarity's sake, the bias terms are not usually included in these types of diagrams. The $\mathbf a$'s are the activation units. $\mathbf a^{(1)}$ is just $\mathbf X$, plus a column of one's for the bias variable. Input 1 is "Height" and Input 2 is "Daily Caloric Intake." In a single logistic regression, $\boldsymbol\theta^{(1)}$ would be $[1 \times 2]$. But, we're doing 5 logistic regressions, one for each neuron in the hidden layer, so $\boldsymbol\theta^{(1)}$ is $[5 \times 2]$.

In [ ]:
n_hl_nodes = 5
np.random.seed(37)
theta1 = np.random.randn(n_hl_nodes, n)

In [ ]:
theta1

In [ ]:
theta1.shape

**Note:** When initializing $\boldsymbol\theta$, it is important to break symmetry. If $\boldsymbol\theta$ is symmetric, then some of the nodes in the hidden layer will turn out to be the same and the algorithm will not be as efficient.

The bias vector is going to be $[1 \times 5]$, one bias term per hidden node/logistic regression.

In [ ]:
b1 = np.random.randn(1, n_hl_nodes)

In [ ]:
b1.shape

Now, $\mathbf a^{(2)}$ is just $g(b+\boldsymbol{x \theta}^T)$.

In [ ]:
# define our activation function
def sigmoid(z):
    g = 1 / (1 + np.exp(-z))
    return g

In [ ]:
a1 = X.copy()

In [ ]:
a1.shape #same as X, check!

In [ ]:
z1 = np.dot(a1, theta1.T) + b1 #broadcasting! :O
a2 = sigmoid(z1)

In [ ]:
a2[:5]

In [ ]:
a2.shape #shape is (m, n_hl_nodes), check!

Essentially, we've turned our 2 feature problem into a 5 feature problem now. The next step is to use $\mathbf a^{(2)}$ like it's our new $\mathbf{X}$.

In [ ]:
n_classes = 2
np.random.seed(42)
theta2 = np.random.randn(n_classes, n_hl_nodes)

In [ ]:
theta2

In [ ]:
theta2.shape

In [ ]:
b2 = np.random.randn(1, n_classes)

In [ ]:
b2.shape

In [ ]:
z2 = np.dot(a2, theta2.T) + b2
a3 = sigmoid(z2)

In [ ]:
a3.shape #shape is (m, n_classes), check!

In [ ]:
a3[:5]

$\mathbf a^{(3)}$ is $ h_{\theta}(x)$. You might also realize it's not the same size as $y$. What we need to do is make a $y_{big}$ matrix that marks a 1 in the row under the column that corresponds to the class. Say our first observation is a hobbit. The first row gets a 1 in the hobbit column. The second observation is not a hobbit, so the second row gets a 1 in the not hobbit column. Since we're doing a binary problem, this works out to $[y, 1-y]$, but it can be generalized to any number of classes.

In [ ]:
y_big = np.column_stack((y, 1-y))

In [ ]:
y_big.shape #same shape as a3, check!

In [ ]:
y_big[:5]

Now to calculate the cost. We need to calculate it per class, which means matrix math won't quite do the trick. We have to loop over the columns. First without regularization.

In [ ]:
J = 0
for i in range(n_classes):
    J += -1 / m * (np.dot(y_big[:,i].T,np.log(a3[:,i])) + np.dot((1 - y_big[:,i]).T, np.log(1 - a3[:,i])))

In [ ]:
J

Now add regularization.

In [ ]:
reg_lambda = 0.01 # remember that lambda is a reserved word

In [ ]:
J += reg_lambda / (2 * m) * (np.dot(theta1.flatten().T, theta1.flatten()) +
                             np.dot(theta2.flatten().T, theta2.flatten()))

In [ ]:
J

### Back propagation
We've got our first predictions. Now we need to compare them to our labels and use this to update our parameters. This isn't quite as simple as just applying your solver to the layers. We'll go through these step by step. The first thing to do is figure out how to calculate our residuals, $\boldsymbol\delta$. $\boldsymbol\delta^{(3)}$ is easy:
$$ \boldsymbol\delta^{(3)} = \mathbf a^{(3)} - \mathbf y_{big} $$

In [ ]:
d3 = a3 - y_big

In [ ]:
d3.shape #same shape as a3 and y_big, check!

In [ ]:
d3[:5]

For the remaining layers, except for the input layer, the residual is calculated as follows:
$$ \boldsymbol\delta^{(l)} = \boldsymbol\delta^{(l+1)} \boldsymbol\theta^{(l)} .* g'(\mathbf z^{(l)}) $$
where $l$ is the layer number and I'm using $.*$ to emphasize that this is element-wise multiplication, not matrix multiplication. Turns out that $g'(\mathbf z^{(l)})$ is just $\mathbf a^{(l)} .* (1 - \mathbf a^{(l)})$, so:
$$ \boldsymbol\delta^{(l)} = \boldsymbol\delta^{(l+1)} \boldsymbol\theta^{(l)} .* \mathbf a^{(l)} .* (1 - \mathbf a^{(l)}) $$

There is no residual for the input layer ($l=1$), so we only have $\boldsymbol\delta^{(2)}$ left to calculate:
$$ \boldsymbol\delta^{(2)} = \boldsymbol\delta^{(3)} \boldsymbol\theta^{(2)} .* \mathbf a^{(2)} .* (1 - \mathbf a^{(2)}) $$

In [ ]:
d2 = np.dot(d3, theta2) * a2 * (1 - a2)

In [ ]:
d2.shape #same shape as a2, check!

In [ ]:
d2[:5]

Now we need to turn this into an overal gradient that we can feed to a solver. First, without regularization, the equation for $\boldsymbol\theta$ is:
$$\dfrac{\partial J(\Theta)}{\partial \Theta_{i,j}^{(l)}} = \frac{1}{m}\sum_{t=1}^m a_j^{(t)(l)} {\delta}_i^{(t)(l+1)}$$

Vectorized:
$$ \dfrac{\partial J(\boldsymbol\Theta)}{\partial \boldsymbol\Theta^{(l)}} = \frac{1}{m} \sum^l \mathbf a^{(l)} \boldsymbol\delta^{(l+1)} $$

In [ ]:
D2 = np.dot(d3.T, a2)

In [ ]:
D2.shape #same shape as theta2, check!

In [ ]:
D2

In [ ]:
D1 = np.dot(d2.T, a1)

In [ ]:
D1.shape #same shape as theta1, check!

In [ ]:
D1

For $b^{(l)}$, it's just $\mathbf{I}^{(l)T}\boldsymbol\delta^{(l+1)}$, where $\mathbf{I}^{(l)}$ is an identity vector with the same number of rows as $\mathbf a^{(l)}$. This works out to summing the columns of $\mathbf a^{(l)}$.

In [ ]:
B2 = np.sum(d3, axis=0, keepdims=True)

In [ ]:
B1 = np.sum(d2, axis=0, keepdims=True)

The regularization term is easy. Just add this:
$$ \frac {\lambda}{2 * m} \boldsymbol\Theta^{(l)} $$
to this:
$$ \dfrac{\partial J(\boldsymbol\Theta)}{\partial \boldsymbol\Theta^{(l)}} $$

In [ ]:
D1 = D1 + theta1 * reg_lambda
D2 = D2 + theta2 * reg_lambda

Remember that the bias terms don't get regularization.

### Optimizing
Now that we have our cost function and our derivative, we can use gradient descent to solve it. Here's what the first update looks like.

In [ ]:
alpha = 0.0005
theta1 += -alpha * D1
theta2 += -alpha * D2
b1 += -alpha * B1
b2 += -alpha * B2

That was easy. Now we just keep going until the cost stops changing by more than our tolerance threshold.

In [ ]:
# let's collect the forward propagation steps
def forward_propagation(a1, theta1, b1, theta2, b2, y_big):
    z1 = np.dot(a1, theta1.T) + b1
    a2 = sigmoid(z1)
    z2 = np.dot(a2, theta2.T) + b2
    a3 = sigmoid(z2)
    J = 0
    for i in range(n_classes):
        J += -1 / m * (np.dot(y_big[:,i].T,np.log(a3[:,i])) + np.dot((1 - y_big[:,i]).T, np.log(1 - a3[:,i])))
        J += reg_lambda / (2 * m) * (np.dot(theta1.flatten().T, theta1.flatten()) + 
                                     np.dot(theta2.flatten().T, theta2.flatten()))
    
    return J

In [ ]:
# let's collect the back propagation steps
def back_propagation(a3, y_big, theta2, a2, a1, theta1, reg_lambda):
    d3 = a3 - y_big
    d2 = np.dot(d3, theta2) * a2 * (1 - a2)
    D1 = np.dot(d2.T, a1)
    D2 = np.dot(d3.T, a2)
    B2 = np.sum(d3, axis=0, keepdims=True)
    B1 = np.sum(d2, axis=0, keepdims=True)
    D1 = D1 + theta1 * reg_lambda
    D2 = D2 + theta2 * reg_lambda
    
    return D1, D2, B1, B2

In [ ]:
tolerance = 0.0001
delta = 1
old_J = J

In [ ]:
J

In [ ]:
#again and again and again
while delta > tolerance:
    #there
    J = forward_propagation(a1, theta1, b1, theta2, b2, y_big)
    
    #and back
    D1, D2, B1, B2 = back_propagation(a3, y_big, theta2, a2, a1, theta1, reg_lambda)
    
    theta1 += -alpha * D1
    theta2 += -alpha * D2
    b1 += -alpha * B1
    b2 += -alpha * B2
    
    delta = old_J - J
    old_J = J
    print(J)

Well, the cost function isn't quite minimized. This is the down side of gradient descent; it tends to overshoot. We could spend a bit more time trying to debug this (which will probably involve rearranging the loop)...or you get the gist of it and we can move on.

### Modification
We made several choices in our implemetation: the activation function, the cost function, and the solver. Any of these can be changed to suit your purposes. For example, you can use the sum of squares as the cost function to use neural nets for regression. The blog post I referenced used a different function in the final layer than for the hidden layers. The catch is that you have to rederive all the propagation steps appropriately...or find someone who's done that already. ;)

## Second breakfast: Prebaked neural nets
### scikit learn
There are quite a few Python modules that can do neural nets. For small problems, scikit learn is more than adequate if you want one stop ML shopping.

In [ ]:
from sklearn.neural_network import MLPClassifier

Here's how to do what we did from scratch.

In [ ]:
clf = MLPClassifier(solver='sgd', alpha=0.01, hidden_layer_sizes=(5,), #alpha is our lambda!
                    activation='logistic', learning_rate_init=0.0005, 
                    random_state=0) #we used several seeds, so I'll use the first one

In [ ]:
# MLPClassifier likes 1D arrays
y = y.reshape(m)
clf.fit(X,y)

Note that there are parameters for the activation function and the solver.

Let's try a more interesting example using the <a href="http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html#sklearn.datasets.load_digits">digits dataset</a>. It consists of pictures of handwritten digits.

In [ ]:
from sklearn.datasets import load_digits
X, y = load_digits(return_X_y=True)
digits = load_digits()
plt.gray() 
plt.matshow(digits.images[0])
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

Oops! Let's change some parameters to try to get it to converge. Let's use the default solver (adam), the default activation function (relu), and two hidden layers with 10 nodes each.

In [ ]:
clf = MLPClassifier(alpha=0.01, hidden_layer_sizes=(10, 10), # the numbers in the tuple are how many nodes in a layer
                    random_state=0)

In [ ]:
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

Not bad!

### Keras
For anything even remotely expensive, I recommend Keras. It's a Python front end for building any neural network architecture you can dream up. There's also a wrapper for it in R called kerasR. On the backend, you can have Tensorflow, CNTK, or Theano. The default backend is Tensorflow because Tensorflow.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.initializers import RandomUniform

We'll do the same NN that we did above for the digits dataset, just for comparison. In deep learning, the type of neural network we've been doing (multi-layer perceptron) is always the final step. It is called the fully connected layer, or the dense layer.

In [ ]:
#make a seed
RandomUniform(seed=0)

In [ ]:
# Convert class vectors to binary class matrices.
n_classes = 10
y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes)

In [ ]:
model = Sequential() #we're going to add all the layers one by one

In [ ]:
#build the layers
model.add(Dense(10, input_shape=X_train.shape[1:])) #1st hidden layer with 10 nodes
model.add(Activation('relu')) #activation function
model.add(Dense(10)) #2nd hidden layer with 10 nodes
model.add(Activation('relu'))
model.add(Dense(n_classes)) #output layer
model.add(Activation('softmax')) #turns out sklearn always uses softmax at the output layer

Fun fact: The default configuration for Adam is actually the same for scikit learn and keras. Turns out they both just stole the default parameter values from the original paper. You can mess with the parameters in both, but I'll just use the defaults.

In [ ]:
#configure the optimizer
opt = Adam()

In [ ]:
# configure the learning process
model.compile(loss='categorical_crossentropy', #this is our logistic regression loss function
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train,
          batch_size=200,
          epochs=200,
          validation_data=(X_test, y_test),
          shuffle=True)

Better than scikit learn!